In [1]:
from tensorflow.contrib.keras.api.keras.layers import Dropout
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Conv2D
from tensorflow.contrib.keras.api.keras.layers import MaxPooling2D
from tensorflow.contrib.keras.api.keras.layers import Flatten
from tensorflow.contrib.keras.api.keras.layers import Dense
from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
import os

In [2]:
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n"\
            .format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1
 
    def on_train_begin(self, logs={}):
        self.losses += 'Training begins...\n'

In [3]:
training_set_path = 'dataset/training_set'
test_set_path = 'dataset/test_set'

In [4]:
# Initialising the CNN
classifier = Sequential()
 
# Step 1 - Convolution
input_size = (128, 128)
classifier.add(Conv2D(32, (3, 3), input_shape=(*input_size, 3), activation='relu'))
 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))  # 2x2 is optimal
 
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=1, activation='sigmoid'))
 
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
# Part 2 - Fitting the CNN to the images
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 class_mode='binary')
 
test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
# Create a loss history
history = LossHistory()
 
classifier.fit_generator(training_set,
                         steps_per_epoch=8000/batch_size,
                         epochs=5, #90
                         validation_data=test_set,
                         validation_steps=2000/batch_size,
                         workers=12,
                         max_queue_size=100,
                         callbacks=[history])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
250/250 [==============================] - 53s 212ms/step - loss: 0.6654 - acc: 0.5928 - val_loss: 0.6202 - val_acc: 0.6420
Epoch 2/5
250/250 [==============================] - 53s 211ms/step - loss: 0.6036 - acc: 0.6750 - val_loss: 0.5904 - val_acc: 0.6850
Epoch 3/5
250/250 [==============================] - 52s 210ms/step - loss: 0.5613 - acc: 0.7072 - val_loss: 0.5583 - val_acc: 0.7020
Epoch 4/5
250/250 [==============================] - 53s 212ms/step - loss: 0.5301 - acc: 0.7349 - val_loss: 0.5362 - val_acc: 0.7505
Epoch 5/5
250/250 [==============================] - 53s 211ms/step - loss: 0.5109 - acc: 0.7560 - val_loss: 0.4791 - val_acc: 0.7800


In [7]:
# Save model
model_backup_path = 'dataset/cat_or_dogs_model.h5'
classifier.save(model_backup_path)
print("Model saved to", model_backup_path)
 
# Save loss history to file
loss_history_path = 'loss_history.log'
myFile = open(loss_history_path, 'w+')
myFile.write(history.losses)
myFile.close()

backend.clear_session()
print("The model class indices are:", training_set.class_indices)

Model saved to dataset/cat_or_dogs_model.h5
The model class indices are: {'cats': 0, 'dogs': 1}
